In [ ]:
import os.path
from os import path
import numpy as np
import matplotlib.pyplot as pyplot
import gc
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import tensorflow as tf
import matplotlib.pyplot as plt
import gc
import statistics
import seaborn as sn
import random
import copy
import math
from itertools import permutations
from scipy.signal import find_peaks

Fill in the following paths that contain:

detector_config_path: constants saved in the detector training\
path_model_NP: Trained NoA-DNN of the Deep Detector\
path_model_NP_params: Trained NoA-DNN of the Deep Detector parameters\
path_model_1P: Trained 1st LoA-DNN of the Deep Detector\
path_model_1P_params: Trained 1st LoA-DNN of the Deep Detector parameters\
path_model_2P: Trained 2nd LoA-DNN of the Deep Detector\
path_model_2P_params: Trained 2nd LoA-DNN of the Deep Detector parameters\
path_model_3P: Trained 3rd LoA-DNN of the Deep Detector\
path_model_3P_params: Trained 3rd LoA-DNN of the Deep Detector parameters\
neutrons_train_path: neutron pulses used for training\
gammas_train_path: gamma pulses used for training\
neutrons_val_path: neutron pulses used for validation\
gammas_val_path: gamma pulses used for validation\
template_n_path: template of the average neutron\
template_g_path: template of the average gamma

Provide the following paths in which values will be saved:

path_model_Attention: Deep Classifier\
path_model_Attention_params: Deep Classifier parameters\
classifier_config_path: constants are saved here

In [ ]:
with open(detector_config_path, "rb") as file:
  loaded_constants = pickle.load(file)

arrival_rate = loaded_constants["arrival_rate"]
g_n_ratio = loaded_constants["g_n_ratio"]
WL_trigger_middle = loaded_constants["WL_trigger_middle"]
margin = loaded_constants["margin"]
num_data = loaded_constants["num_data"]
starting_index = loaded_constants["starting_index"]
template_length_orig_features = loaded_constants["template_length_orig_features"]
Ts = loaded_constants["Ts"]
num_original_features = loaded_constants["num_original_features"]
num_classification_features = loaded_constants["num_classification_features"]
num_attention_features = loaded_constants["num_attention_features"]
new_starting_point = loaded_constants["new_starting_point"]
pulse_len_after_starting = loaded_constants["pulse_len_after_starting"]
time_offset = loaded_constants["time_offset"]
success_marg = loaded_constants["success_marg"]
L_deriv_orig_features = loaded_constants["L_deriv_orig_features"]
delta_orig_features = loaded_constants["delta_orig_features"]
short_gate_class_features = loaded_constants["short_gate_class_features"]
short_gate_orig_features = loaded_constants["short_gate_orig_features"]
long_gate_class_features = loaded_constants["long_gate_class_features"]
long_gate_orig_features = loaded_constants["long_gate_orig_features"]
max_random_offset = loaded_constants["max_random_offset"]
cont_starting_time = loaded_constants["cont_starting_time"]
original_pulse_len = loaded_constants["original_pulse_len"]
std_noise_total = loaded_constants["std_noise_total"]
std_noise_curr_constant = loaded_constants["std_noise_curr_constant"]
noise_mean = loaded_constants["noise_mean"]
low_NN = loaded_constants["low_NN"]
high_NN = loaded_constants["high_NN"]
len_true_inf = loaded_constants["len_true_inf"]
WL_trigger = loaded_constants["WL_trigger"]
lambda_n = loaded_constants["lambda_n"]
lambda_g = loaded_constants["lambda_g"]
std_noise_curr = loaded_constants["std_noise_curr"]
std_added_noise = loaded_constants["std_added_noise"]
jump = loaded_constants["jump"]
arrival_rate_text = loaded_constants["arrival_rate_text"]
curr_path = loaded_constants["curr_path"]
model_NP_layer1 = loaded_constants["model_NP_layer1"]
model_NP_layer2 = loaded_constants["model_NP_layer2"]
model_NP_layer3 = loaded_constants["model_NP_layer3"]
model_NP_layer4 = loaded_constants["model_NP_layer4"]
model_NP_layer5 = loaded_constants["model_NP_layer5"]
model_P3_layer5 = loaded_constants["model_P3_layer5"]
model_P2_layer5 = loaded_constants["model_P2_layer5"]
model_P2_layer6 = loaded_constants["model_P2_layer6"]
model_P1_layer5 = loaded_constants["model_P1_layer5"]
model_P1_layer6 = loaded_constants["model_P1_layer6"]
model_NP_lr = loaded_constants["model_NP_lr"]
model_NP_bs = loaded_constants["model_NP_bs"]
model_NP_epochs = loaded_constants["model_NP_epochs"]
model_3P_lr = loaded_constants["model_3P_lr"]
model_3P_bs = loaded_constants["model_3P_bs"]
model_3P_epochs = loaded_constants["model_3P_epochs"]
model_2P_lr = loaded_constants["model_2P_lr"]
model_2P_bs = loaded_constants["model_2P_bs"]
model_2P_epochs = loaded_constants["model_2P_epochs"]
model_1P_lr = loaded_constants["model_1P_lr"]
model_1P_bs = loaded_constants["model_1P_bs"]
model_1P_epochs = loaded_constants["model_1P_epochs"]

run <your-path> 'General_Functions.ipynb>
run <your-path> 'Deep_Detector_Functions.ipynb>
run <your-path> 'Deep_Detector_Networks.ipynb>

# Load the Deep Detector models
model_trigger_classification = torch.load(path_model_NP)
norm_params_model_trigger_classification = np.load(path_model_NP_params)

model_arrival_times_1 = torch.load(path_model_1P)
norm_params_model_arrival_times_1 = np.load(path_model_1P_params)

model_arrival_times_2 = torch.load(path_model_2P)
norm_params_model_arrival_times_2 = np.load(path_model_2P_params)

model_arrival_times_3 = torch.load(path_model_3P)
norm_params_model_arrival_times_3 = np.load(path_model_3P_params)

Num_train = 100000 # Number of pulses used for training
Num_val = 40000 # Number of pulses used for validation
Num_test = 60000 # Number of pulses used for testing
WL_classifier = 200 # Number of pulse samples used for DNN methods

neutrons_train = np.load(neutrons_train_path)
gammas_train = np.load(gammas_train_path)
neutrons_val = np.load(neutrons_val_path)
gammas_val = np.load(gammas_val_path)
template_n = np.load(template_n_path)
template_g = np.load(template_g_path)

train_data, train_labels = create_classifier_data_for_training(neutrons_train,gammas_train,Num_train,False)

# Deep Classifier Layers
model_classifier_layer1 = 130
model_classifier_layer2 = 80
model_classifier_layer3 = 30
model_classifier_layer4 = 12
model_classifier_layer5 = 4
model_attention_layer1 = 30
model_attention_layer2 = 80

run <your-path> 'Classification_Algorithm_Functions.ipynb>

CI_param1_lst = range(1,100,1) # Short gate length in samples
CI_param2_lst = range(1,200,1) # Long gate length in samples
CI_param3_lst = range(-30,50,1) # Delay before starting to integrate in samples
CI_param4_lst = list(np.linspace(0.2,1,200)) # Threshold
CI_parameters = optimize_charge_integration_parameters(train_data,train_labels)
np.save(path_CI_parameters,CI_parameters)
corr_param1_lst = range(0,30,1) # Correlation starting index
corr_param2_lst = range(30,200,1) # Correlation ending index
Corr_parameters = optimize_correlation_parameters(train_data,train_labels)
np.save(path_NCC_classifier_parameters,Corr_parameters)

train_data, train_labels, train_class_feat, train_weigh_feat = create_classifier_data_for_training(neutrons_train,gammas_train,Num_train,True)
val_data, val_labels, val_class_feat, val_weigh_feat = create_classifier_data_for_training(neutrons_val,gammas_val,Num_val,True)
short_gate_class_features = CI_parameters[0]
long_gate_class_features = CI_parameters[1]
start_gate_class_features = CI_parameters[2]
corr_start_class_features = Corr_parameters[0]
corr_end_class_features = Corr_parameters[1]

## Training the Deep Classifier
model_Attention_lr = 1e-3
model_Attention_bs = 30
model_Attention_epochs = 70
run_Model_Attention(train_labels,train_data,val_labels,val_data,
                val_labels,val_data,train_class_feat,val_class_feat,val_class_feat,
                train_weigh_feat,val_weigh_feat,val_weigh_feat,path_model_Attention,path_model_Attention,
                path_model_Attention_params, model_Attention_lr, model_Attention_bs, model_Attention_epochs)

constants_dict = {

    "short_gate_class_features": short_gate_class_features,
    "long_gate_class_features": long_gate_class_features,
    "start_gate_class_features": start_gate_class_features,
    "corr_start_class_features": corr_start_class_features,
    "corr_end_class_features": corr_end_class_features,

    "Num_train": Num_train,
    "Num_val": Num_val,
    "Num_test": Num_test,
    "Num_BM": Num_BM,
    "WL_classifier": WL_classifier,

    "model_classifier_layer1": model_classifier_layer1,
    "model_classifier_layer2": model_classifier_layer2,
    "model_classifier_layer3": model_classifier_layer3,
    "model_classifier_layer4": model_classifier_layer4,
    "model_classifier_layer5": model_classifier_layer5,
    "model_attention_layer1": model_attention_layer1,
    "model_attention_layer2": model_attention_layer2,

    "model_Attention_lr": model_Attention_lr,
    "model_Attention_bs": model_Attention_bs,
    "model_Attention_epochs": model_Attention_epochs,
}

with open(classifier_config_path, "wb") as file:
    pickle.dump(constants_dict, file)